# Path

In [11]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Import

In [12]:
from helpers import column_formatter, utf8_convert
from charset_normalizer import detect
from config import data_path as path
from datetime import datetime
from json import dump, load
import pandas as pd
import openpyxl

# Read

In [13]:
lesson_all = pd.read_excel(
    path+'/2-cleaned/basvuru.xlsx', sheet_name='Dersler')
roll_called_lessons = pd.read_excel(
    path+'/2-cleaned/yoklama.xlsx', index_col='Unnamed: 0', sheet_name='General Info')

# Merge & Concat

In [14]:
lesson_all.dropna(inplace=True)
lesson_all.index = lesson_all[0]
lesson_all = lesson_all.T
lesson_all = lesson_all.iloc[:1]
lesson_all.index = ['Senkron Adı']
lesson_all=lesson_all.T

In [15]:
roll_called_lessons=roll_called_lessons.merge(lesson_all, right_index=True, left_index=True)

In [16]:
index = list(set(lesson_all.index)-set(roll_called_lessons.index))

In [17]:
empty_df = pd.DataFrame(columns = roll_called_lessons.columns, index=index)
empty_df['Senkron Adı'] = lesson_all.T[index].T
concat_data = pd.concat([roll_called_lessons, empty_df])

In [18]:
lessons = concat_data.sort_values('TARİH')
lessons_format = pd.DataFrame(lessons['Senkron Adı'], index=lessons.index)

lessons['Normal Adı'] = lessons.index
lessons.index = list(range(lessons.shape[0]))
lessons.drop(7, axis=0, inplace=True)

lessons.index = lessons['Normal Adı']
lessons = lessons[lessons.columns[:-2]]
lessons.index.name = None

lessons['EĞİTİMCİ'] = column_formatter(lessons['EĞİTİMCİ'])

lessons['SÜRE'] = (lessons['BİTİŞ SAATİ']-lessons['BAŞLANGIÇ SAATİ'])
lessons.drop('BİTİŞ SAATİ', axis=1, inplace=True)

lessons = lessons.rename(columns={'BAŞLANGIÇ SAATİ': 'SAAT', 'EĞİTİMCİ':'egitimci'})

lessons.columns = list(map(lambda x: x.lower(), list(lessons.columns)))

lessons['eğitim'] = lessons.index
lessons.index = list(range(len(lessons)))

In [19]:
lessons

,tari̇h,gün,egitimci,saat,ay,süre,eğitim
0,2022-05-25 00:00:00,çarşamba,josephcohen,14.0,mayıs,4.0,Hikaye Anlatıcılığı 101
1,2022-06-02 00:00:00,perşembe,davidgullett,16.0,haziran,4.0,Finans 101
2,2022-06-04 00:00:00,cumartesi,johnweis,10.0,haziran,5.0,Büyük Veriye Giriş 101
3,2022-06-07 00:00:00,salı,shirleylangford,14.0,haziran,3.0,Proje Döngüsü Yönetimi 101
4,2022-06-09 00:00:00,perşembe,adamfreeman,16.0,haziran,4.0,Girişimcilik 101
5,2022-06-11 00:00:00,cumartesi,fredapicciano,10.0,haziran,5.0,Arduino 101
6,2022-06-16 00:00:00,perşembe,carolinaellington,16.0,haziran,4.0,İşletim Sistemleri 101
7,2022-06-18 00:00:00,cumartesi,emmaulrich,10.0,haziran,4.0,IOT 101
8,2022-06-21 00:00:00,salı,harlansippel,14.0,haziran,4.0,Zihin Haritalama 101
9,2022-06-23 00:00:00,perşembe,christinependleton,16.0,haziran,3.0,Blockchain 101


# Write

In [20]:
with pd.ExcelWriter(path+'/3-result/events.xlsx') as writer:
    lessons.to_excel(writer, sheet_name='Events')
    lessons_format.to_excel(writer, sheet_name='Format')